# tensorflow笔记（三）之 tensorboard的使用

## 前言

这篇notebook将介绍tensorflow当中一个非常有用的可视化工具tensorboard的使用，它将对我们分析训练效果，理解训练框架和优化算法都有很大的帮助。

由于很多显示的部分是在ubuntu终端还有代理服务器网页上的，所以jupyter notebook部分只会贴本篇的代码，详细的还请看我的博客

还记得我的第一篇tensorflow博客上的的图片吗？先画出第一篇tensorflow博客的tensorboard图吧

## 1. 矩阵相乘实例

相应的代码

In [4]:
import tensorflow as tf

with tf.name_scope('graph') as scope:
     matrix1 = tf.constant([[3., 3.]],name ='matrix1')  #1 row by 2 column
     matrix2 = tf.constant([[2.],[2.]],name ='matrix2') # 2 row by 1 column
     product = tf.matmul(matrix1, matrix2,name='product')
  
sess = tf.Session()

writer = tf.summary.FileWriter("logs/", sess.graph)

init = tf.global_variables_initializer()

sess.run(init)

In [6]:
pwd

u'/home/fangyang'

重要：运行上面的代码，查询当前目录，就可以找到一个新生成的文件，已命名为logs，我们需在终端上运行tensorboard，生成本地链接，具体看我截图，当然你也可以将上面的代码直接生成一个py文档在终端运行，也会在终端当前目录生成一个logs文件，然后运行tensorboards --logdir logs指令，就可以生成一个链接，复制那个链接，在google浏览器（我试过火狐也行）粘贴显示，对于tensorboard 中显示的网址打不开的朋友们, 请使用 http://localhost:6006 （如果这个没有成功，我之前没有安装tensorboard，也出现链接，但那个链接点开什么都没有，所以还有一种可能就是你没有安装tensorboard，使用pip install tensorboard安装tensorboard，python3用pip3 install tensorboard）

如果出现了我博客上显示出的网页界面，那就证明tensorboard运行成功

## 2. 线性拟合

上面那一个比较简单，没有任何训练过程，下面将第一篇tensorflow笔记中的第二个例子来画出它的流动图（哦，对了，之所有说是流动图，这是由于tensorflow的名字就是向量在图形中流动的意思）

代码如下：

In [7]:
import tensorflow as tf
import numpy as np

## prepare the original data
with tf.name_scope('data'):
     x_data = np.random.rand(100).astype(np.float32)
     y_data = 0.3*x_data+0.1
##creat parameters
with tf.name_scope('parameters'):
     weight = tf.Variable(tf.random_uniform([1],-1.0,1.0))
     bias = tf.Variable(tf.zeros([1]))
##get y_prediction
with tf.name_scope('y_prediction'):
     y_prediction = weight*x_data+bias
##compute the loss
with tf.name_scope('loss'):
     loss = tf.reduce_mean(tf.square(y_data-y_prediction))
##creat optimizer
optimizer = tf.train.GradientDescentOptimizer(0.5)
#creat train ,minimize the loss 
with tf.name_scope('train'):
     train = optimizer.minimize(loss)
#creat init
with tf.name_scope('init'): 
     init = tf.global_variables_initializer()
##creat a Session 
sess = tf.Session()
##initialize
writer = tf.summary.FileWriter("logs/", sess.graph)
sess.run(init)
## Loop
for step  in  range(101):
    sess.run(train)
    if step %10==0 :
        print step ,'weight:',sess.run(weight),'bias:',sess.run(bias)
    

0 weight: [ 0.36441842] bias: [ 0.08811751]
10 weight: [ 0.327126] bias: [ 0.08635258]
20 weight: [ 0.31309825] bias: [ 0.09341011]
30 weight: [ 0.30632472] bias: [ 0.09681796]
40 weight: [ 0.303054] bias: [ 0.09846351]
50 weight: [ 0.30147469] bias: [ 0.09925807]
60 weight: [ 0.30071208] bias: [ 0.09964176]
70 weight: [ 0.30034381] bias: [ 0.09982704]
80 weight: [ 0.30016604] bias: [ 0.09991647]
90 weight: [ 0.30008018] bias: [ 0.09995967]
100 weight: [ 0.30003873] bias: [ 0.09998053]


运行过程我会在博客中贴出

然后我们在对上面的代码进行修改修改，试试tensorboard的其他功能，例如scalars，distributions，histograms,它们对我们分析学习算法的性能有很大帮助

代码如下

In [8]:
import tensorflow as tf
import numpy as np

## prepare the original data
with tf.name_scope('data'):
     x_data = np.random.rand(100).astype(np.float32)
     y_data = 0.3*x_data+0.1
##creat parameters
with tf.name_scope('parameters'):
     with tf.name_scope('weights'):
      	  weight = tf.Variable(tf.random_uniform([1],-1.0,1.0))
     	  tf.summary.histogram('weight',weight)
     with tf.name_scope('biases'):
     	  bias = tf.Variable(tf.zeros([1]))
     	  tf.summary.histogram('bias',bias)
##get y_prediction
with tf.name_scope('y_prediction'):
     y_prediction = weight*x_data+bias
##compute the loss
with tf.name_scope('loss'):
     loss = tf.reduce_mean(tf.square(y_data-y_prediction))
     tf.summary.scalar('loss',loss)
##creat optimizer
optimizer = tf.train.GradientDescentOptimizer(0.5)
#creat train ,minimize the loss 
with tf.name_scope('train'):
     train = optimizer.minimize(loss)
#creat init
with tf.name_scope('init'): 
     init = tf.global_variables_initializer()
##creat a Session 
sess = tf.Session()
#merged
merged = tf.summary.merge_all()
##initialize
writer = tf.summary.FileWriter("logs/", sess.graph)
sess.run(init)
## Loop
for step  in  range(101):
    sess.run(train)
    rs=sess.run(merged)
    writer.add_summary(rs, step)

闲麻烦，我把打印的去掉了。具体图形化我会在博客上展示